# Titanic Survival Prediction: 02 - Modeling
*Date: 14.09.2025*
*Author: Jonas Lilletvedt*

--- 

## 1. Introduction

### 1.1. Objective

This notebook marks the second part of the Titanic Survival Prediction project: model training and evaluation. We will build directly on the pre-processing pipeline constructed in `01_data_cleaning_and_feature_engineering.ipynb`. The primary objective is to systematically train, evaluate, and tune a selection of models to find the optimal balance between predictive performance and interpretability. 

1. **Preparation and Pipeline Reconstruction:** We will begin by loading datasets and rebuilding the pre-processing pipeline from the previous notebook to ensure a consistent and reproducible environment.
2. **Baseline Model Evaluation:** A selection of diverse classification models (e.g., K-NN, Random Forest, Gradient Boosting) will be used as baseline and later help us identify strategies for optimization. 
3. **Hyperparameter Tuning:** The best-performing model from the baseline evaluation will undergo systematic hyperparameter tuning using GridSearchCV. The goal is to maximize its predictive performance based on our primary evaluation metric, the F1-Score. The reasoning for selecting this metric is detailed in the section below.
4. **Final Analysis and Iteration:** Finally, we will analyze the results and train the optimized model on the full dataset to generate final predictions for a Kaggle submission. We will conclude by exploring potential avenues for future improvements, such as alternative modeling techniques or further feature engineering.

Our end goal is twofold: to develop a model that is competitive on the Kaggle leaderboard, while remaining understandable enough to provide insights into the factors that influence survival. The process will be iterative, allowing us to refine our approach based on model performance. 

Beyond the Leaderboard: A Focus on Interpretability
Unlike many competition-driven projects that prioritize raw predictive power above all else, this analysis places a strong emphasis on interpretability. While achieving a high Kaggle score is a key objective, it is equally important to build a model whose decision-making process can be understood. By favoring models and features that provide clear insights—for instance, by showing why certain passengers were more likely to survive—we aim to produce not just a "black box" predictor, but a meaningful analysis of a historical event. This balanced approach ensures that our final result is both powerful and insightful.

### 1.2 Defining Success: Beyond Simple Accuracy

While accuracy is a straightforward metric, it can be misleading. A simple "naive" model that predicts survival for all females and death for all males achieves an accuracy of approximately 78.7% on the training data. Any model we build must therefore significantly outperform this baseline to be considered valuable.

To truly understand our model's performance, we must select the right evaluation metrics for the task. The most useful evaluation metric depends on the specific problem you are trying to solve. For example, in medical screening for a virus, achieving a high Recall is more important than high Precision. We would want to identify as many infected people as possible, even if it means accepting some false positives. In other situations, however, overall Accuracy might be the primary goal.
Applying this to the Titanic problem, our context is one of historical analysis and balanced prediction. There is no real-world cost that makes predicting a death incorrectly worse than predicting a survival incorrectly. Our goal is not just to be right, but to build a model that is equally effective at identifying both those who survived and those who did not.
For this reason, we need metrics that reward this balance and are not skewed by the class distribution. While we will still consider overall Accuracy, our primary metric for model evaluation is:
*   F1-Score: The harmonic mean of precision and recall. Provides an overall score between precision and recall. 

## 2. Data Loading and Setup

---

### 2.1. Library Imports and Seed

In [138]:
# Import necessary libraries

# Data manipulation and analysis
import numpy as np
import pandas as pd

# Set seed for reproducible results
seed = 43

### 2.2. Load Datasets

In [139]:
# Load datasets
df_train = pd.read_csv('../data/01_raw/train.csv')
df_test = pd.read_csv('../data/01_raw/test.csv', index_col='PassengerId')

### 2.3. Initial Inspection

A quick inspection to check the datasets are loaded properly and as expected.

**Datasets Shape:**

In [140]:
# Check shape of each dataset
print(f'Training data shape: {df_train.shape}')
print(f'Test data shape: {df_test.shape}')

Training data shape: (891, 12)
Test data shape: (418, 10)


**Data Preview:**

In [141]:
# Check five first rows in df_train
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [142]:
# Check five first rows in df_test
df_test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


### 2.4. Prepare Data for Modeling

The datasets are loaded as expected. We will now separate our training data into two distinct objects:
*   **X:** A DataFrame containing all predictor variables.
*   **y:** A Series containing the target variable `Survived`, which we aim to predict.

In [143]:
# Seperate predictors from target
X_train = df_train.drop('Survived', axis=1)
y_train = df_train['Survived']

To avoid making changes on the original test data we will copy it to a separate object.

In [144]:
# Copy to avoid making changes to the original dataset
X_test = df_test.copy()

Display the shapes of our new variables to ensure the separation and copy worked as intended.

In [145]:
print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of y_train: {y_train.shape}')
print(f'Shape of X_test: {X_test.shape}')

Shape of X_train: (891, 11)
Shape of y_train: (891,)
Shape of X_test: (418, 10)


### 2.5. Reconstruct the Pre-processing Pipeline

To make this notebook self-contained and reproducible, we will now redefine the custom transformers and the full `grand_pipeline` that were built and validated in the previous notebook. All the data transformation logic is encapsulated within a single code cell below. 

**A Note on Modularity vs. Narrative Flow**

In a production-level software project, this logic would typically be defined in a separate python-file to be imported. However, for this project, which is designed to be a linear, narrative-driven analysis, I have chosen to explicitly include the code here. This approach ensures that the notebook tells the complete, end-to-end story. Any future changes or improvements made in later stages of the project does not affect the logic or result of previous notebooks, preserving the integrity of each step of of our analysis. Furthermore, it allows any modifications or improvements to be documented and explained at the precise moment they are introduced, for a more natural progression.

All data transformation logic is therefore encapsulated within the single code cell below, which can be collapsed for easier reading of the modeling work that follows.

In [146]:
# Scikit-learn tools for preprocessing and modeling
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin

class TitleExtractor(BaseEstimator,  TransformerMixin):
    def __init__(self, rare_threshold=10):
        self.rare_threshold = rare_threshold

        # Titles with same meaning
        self.title_synonym_mapping_ = {
            'Mlle.': 'Miss.',
            'Ms.': 'Miss.',
            'Mme.': 'Mrs.'
        }

    def fit(self, X, y=None):
        titles = X['Name'].str.extract(pat=' ([A-Za-z]+\.)', expand=False)
        titles = titles.replace(self.title_synonym_mapping_)
        self.non_rare_titles_ = titles.value_counts()[lambda x: x >= self.rare_threshold].index
        return self
    
    def transform(self, X, y=None):
        # Copy to avoid modifying original
        X_copy = X.copy()
        # Extract title 
        X_copy['Title_feat'] = X_copy['Name'].str.extract(pat=' ([A-Za-z]+\.)', expand=False)
        
        # Swap titles with 'Rare' and synonym
        X_copy['Title_feat'] = X_copy['Title_feat'].replace(self.title_synonym_mapping_)


        X_copy['Title_feat'] = X_copy['Title_feat'].apply(lambda x: x if x in self.non_rare_titles_ else 'Rare')

        return X_copy

class FamilySurvialRateExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, drop_surname=True, smooth_factor=1):
        self.drop_surname = drop_surname
        self.smooth_factor = smooth_factor

    def fit(self, X, y=None):
        if y is None:
            raise ValueError('''The FamilySurvialRateExtractor requires target variable 'y' for fitting''')

        X_temp = pd.concat([X, y], axis=1)
        X_temp['FamilyID_temp'] = self.__get_family_id(X)


        self.family_stats_ = X_temp.groupby('FamilyID_temp').agg(
            FamilySize_temp = ('Survived', 'size'), 
            FamilySurvivalCount_temp = ('Survived', 'sum')
        )
    
        self.global_survival_rate_ = y.mean()
        self.training_index_ = X.index
        self.y_train_ = y

        solo_travelers_stats = self.family_stats_[self.family_stats_['FamilySize_temp'] == 1]
        
        if not solo_travelers_stats.empty:
            self.alone_survival_rate_ = solo_travelers_stats['FamilySurvivalCount_temp'].mean()
        else:
            self.alone_survival_rate_ = self.global_survival_rate_

        return self

    def transform(self, X, y=None):
        X_copy = X.copy()

        X_copy['FamilyID_temp'] = self.__get_family_id(X)

        X_copy = X_copy.merge(self.family_stats_, on='FamilyID_temp', how='left')

        # Different calculation for passengers from training set -- avoid data leakage
        if X.index.equals(self.training_index_):
            X_copy['FamilySurvivalCount_temp'] -= self.y_train_
            X_copy['FamilySize_temp'] -= 1

        numerator = X_copy['FamilySurvivalCount_temp'] 
        denominator = X_copy['FamilySize_temp']
        
        # Apply smoothing
        smoothed_numerator = numerator + (self.smooth_factor * self.global_survival_rate_)
        smoothed_denominator = denominator + self.smooth_factor

        # Calculate FamilySurvivalRate
        X_copy['FamilySurvivalRate_feat'] = smoothed_numerator / smoothed_denominator

        # For passengers without a family from training
        X_copy['FamilySurvivalRate_feat'] = X_copy['FamilySurvivalRate_feat'].fillna(self.global_survival_rate_)

        # For passengers which travel alone we will overwrite global_survival_rate_
        is_alone_mask = (X_copy.groupby('FamilyID_temp')['FamilyID_temp'].transform('count') == 1) & (X_copy['FamilySize_temp'] == 0 | X_copy['FamilySize_temp'].isna())

        X_copy.loc[is_alone_mask, 'FamilySurvivalRate_feat'] = self.alone_survival_rate_

        # Columns to drop
        columns_to_drop = ['FamilySurvivalCount_temp', 'FamilySize_temp', 'FamilyID_temp']
        
        # Clean df
        X_copy = X_copy.drop(columns_to_drop, axis=1)

        return X_copy

    def __get_surname(self, X):
        # Extract surname
        data = X.copy()
        return data['Name'].str.extract(pat=r'^(.+)?,', expand=False)
    
    def __get_family_id(self, X):
        return self.__get_surname(X) + '_' + X['Pclass'].astype(str)

class AgeImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        X_temp = X.copy()
        self.median_by_title_ = X_temp.groupby('Title_feat')['Age'].median()
        return self
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        X_copy['Age'] = X_copy['Age'].fillna(X_copy['Title_feat'].map(self.median_by_title_))
        return X_copy

class CabinLocationExtractor(BaseEstimator,  TransformerMixin):
    def __init__(self, drop_original=True):
        self.drop_original = True
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_copy = X.copy()
        X_copy['Deck_feat'] = X_copy['Cabin'].str.extract(pat=r'^([A-Za-z])?', expand=False).fillna('U')
        X_copy['Zone_feat'] = pd.to_numeric(
            X_copy['Cabin'].str.extract(pat=r'([0-9]+)', expand=False),
            errors='coerce'
            )
        
        if self.drop_original:
            X_copy = X_copy.drop('Cabin', axis=1)

        return X_copy
    
feature_engineering_pipeline = Pipeline(steps=[
    ('family_survival_rate_extractor', FamilySurvialRateExtractor()),
    ('title_extractor', TitleExtractor()),
    ('age_imputer', AgeImputer()),
    ('cabin_location_extractor', CabinLocationExtractor())
])

FARE_TO_LOG_TRANS = ['Fare']
FAMILY_SURVIVAL_RATE_FEAT = ['FamilySurvivalRate_feat']
CAT_FEATURES = ['Sex',
                        'Embarked',
                        'Pclass',
                        'Title_feat',
                        'Deck_feat'
                        ]
AGE_TO_BIN = ['Age']
ZONE_TO_BIN = ['Zone_feat']


fare_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('log_transform', FunctionTransformer(np.log1p, feature_names_out='one-to-one')),
    ('scaler', StandardScaler())
])

family_survival_rate_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Simple function for pd.cut
def apply_pd_cut(X, bins, labels):
    series = pd.Series(np.array(X).ravel()) 
    binned_series = pd.cut(series, bins=bins, labels=labels, right=True, include_lowest=True)
    return binned_series.to_numpy().reshape(-1, 1)

# Bins for Age
# Infant: 0-5, Child: 6-12, young-adult: 13-25, adult: 26-50, elder: 51->
AGE_BINS = [0, 5, 12, 25, 50, np.inf]
AGE_LABELS = ['Infant', 'Child', 'Young-Adult', 'Adult', 'Senior']

def age_binner_feature_name(_, input_features):
    return ['Age_binned']

age_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('binner', FunctionTransformer(apply_pd_cut, kw_args={'bins': AGE_BINS, 'labels': AGE_LABELS}, feature_names_out=age_binner_feature_name)),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

def zone_binner_feature_name(_, input_features):
    return ['Zone_binned']

def apply_bin_zone(X, n_bins):
    series = pd.Series(np.array(X).ravel()) 
    labels  = [f'Q{i}' for i in range(1, n_bins + 1)]
    binned_series = pd.qcut(series, q=n_bins, labels=labels, duplicates='drop')

    # Convert nans to unknown
    binned_series = binned_series.cat.add_categories(['Unknown']).fillna('Unknown')
    return binned_series.to_numpy().reshape(-1, 1)

zone_pipeline = Pipeline(steps=[
    ('bin_with_missing_values', FunctionTransformer(apply_bin_zone, kw_args={'n_bins': 8}, feature_names_out=zone_binner_feature_name)),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('fare_log', fare_pipeline, FARE_TO_LOG_TRANS),
        ('family_survival_rate', family_survival_rate_pipeline, FAMILY_SURVIVAL_RATE_FEAT),
        ('cat', categorical_pipeline, CAT_FEATURES),
        ('age_binned', age_pipeline, AGE_TO_BIN),
        ('zone_binned', zone_pipeline, ZONE_TO_BIN)
    ],
    remainder='drop'
)

grand_pipeline = Pipeline(steps=[
    ('feature_engineering', feature_engineering_pipeline),
    ('preprocessing', preprocessor)
])  


## 3. Baseline Model Evaluation

---
With our pre-processing pipeline fully constructed, we can now proceed to the modeling phase. The first step is to establish a performance baseline by evaluating several different classification algorithms. This process will help us identify the most promising model architecture before investing time in hyperparameter tuning.

As established in the introduction, our primary metric for model selection will be the F1-Score, as it provides a balanced measure of a model's performance. For a more complete picture, we will also evaluate Precision, Recall, and overall Accuracy.

To obtain a reliable estimate of each model's generalization performance, we will use 5-fold cross-validation. This method helps mitigate the risk of overfitting to a single train-test split and provides a more robust foundation for our subsequent decisions.

For our baseline we will use four different classification models. 
1. **Logistic Regression**
2. **K-Nearest Neighbors**
3. **Random Forest**
4. **Gradient Boosting**
5. **Support Vector Machine**

In [147]:
# Removing Warnings -- Make output hard to read
# We get a future warning from pipeline in the code block below. We have used the pipeline and functions as intended, I am therefore choosing to ignore them.
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [148]:
# Import SKlearn models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

# Define models
models = {
    'Logistic Regression': LogisticRegression(random_state=seed, max_iter=1000),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(random_state=seed),
    'Gradient Boosting': GradientBoostingClassifier(random_state=seed), 
    'Support Vector Machine': SVC(random_state=seed, probability=True)
}

scoring_metrics = ['f1', 'accuracy']

# Dict for results
results = {}

print('Starting baseline-evaluation of models...\n')

# Iterate over all models
for model_name, model in models.items():

    # Train model using the pre-constructed pipeline
    full_pipeline = Pipeline(steps=[
        ('preprocessor', grand_pipeline),
        ('classifier', model)
    ])

    cv_results = cross_validate(
        full_pipeline, 
        X_train, 
        y_train, 
        cv=5, 
        scoring=scoring_metrics,
        n_jobs=1,  
        error_score='raise'
    )

    # Store and print the results for both metrics
    mean_f1 = np.mean(cv_results['test_f1'])
    std_f1 = np.std(cv_results['test_f1'])
    mean_accuracy = np.mean(cv_results['test_accuracy'])
    std_accuracy = np.std(cv_results['test_accuracy'])
    
    results[model_name] = {
        'mean_f1': mean_f1, 'std_f1': std_f1,
        'mean_accuracy': mean_accuracy, 'std_accuracy': std_accuracy
    }

    print(f'''{model_name}: Mean F1-Score = {results[model_name]['mean_f1']:.4f}''')
    print(f'''{model_name}: Std-F1-Score = {results[model_name]['std_f1']:.4f}''')
    print(f'''{model_name}: Mean Accuracy Score = {results[model_name]['mean_accuracy']:.4f}''')
    print(f'''{model_name}: Std-Accuracy Score = {results[model_name]['std_accuracy']:.4f}\n''')

best_model_name = max(results, key=lambda k: results[k]['mean_f1'])

print(f'''Best performing model: {best_model_name} with a mean F1-Score of {results[best_model_name]['mean_f1']:.4f}''')


Starting baseline-evaluation of models...

Logistic Regression: Mean F1-Score = 0.7617
Logistic Regression: Std-F1-Score = 0.0206
Logistic Regression: Mean Accuracy Score = 0.8193
Logistic Regression: Std-Accuracy Score = 0.0146

K-Nearest Neighbors: Mean F1-Score = 0.7431
K-Nearest Neighbors: Std-F1-Score = 0.0560
K-Nearest Neighbors: Mean Accuracy Score = 0.8137
K-Nearest Neighbors: Std-Accuracy Score = 0.0384

Random Forest: Mean F1-Score = 0.7528
Random Forest: Std-F1-Score = 0.0410
Random Forest: Mean Accuracy Score = 0.8205
Random Forest: Std-Accuracy Score = 0.0189

Gradient Boosting: Mean F1-Score = 0.7595
Gradient Boosting: Std-F1-Score = 0.0209
Gradient Boosting: Mean Accuracy Score = 0.8272
Gradient Boosting: Std-Accuracy Score = 0.0039

Support Vector Machine: Mean F1-Score = 0.7647
Support Vector Machine: Std-F1-Score = 0.0392
Support Vector Machine: Mean Accuracy Score = 0.8361
Support Vector Machine: Std-Accuracy Score = 0.0176

Best performing model: Support Vector Mach

The baseline evaluation identifies the **Support Vector Machine (SVM)** as the best performing model in both key metrics:

*   **Mean F1-Score:** 0.7647
*   **Mean Accuracy:** 0.8361


Gradient Boosting is a very close second (0.7595 F1-Score), both looks like promising candidates for further improvements.

For now we will move forward with SVM and Gradient Boosting as models of choice for hyperparameter tuning.

## 4. Hyperparameter Tuning

---

The next step is to tune the Support Vector Machine in hopes of further improving its performance. We will use GridSearchCV to systematically test different combinations of the model's key parameters to find the optimal combination for our data.

### 4.1. Hyperparameter Tuning Support Vector Machine

In [149]:
from sklearn.model_selection import GridSearchCV

# Best performing model from base evaluation
svc = SVC(random_state=seed, probability=True)

# Create tuning pipeline
tuning_pipeline = Pipeline(steps=[
    ('preprocessor', grand_pipeline),
    ('classifier', svc)
])

# Define parameter grid
param_grid = {
    'classifier__C': [0.1, 1, 10, 50, 100],
    'classifier__gamma': ['scale', 'auto', 0.01, 0.1, 1], 
    'classifier__kernel': ['rbf']
}

# Create and run GridSearchCV object
grid_search_svm = GridSearchCV(
    tuning_pipeline, 
    param_grid,
    cv=5, 
    scoring={'f1': 'f1', 'accuracy': 'accuracy'},
    refit='f1',
    n_jobs=1
)

# Terminal comments
print('Starting Hyperparameter Tuning for Support Vector Machine...')

# Run tuning
grid_search_svm.fit(X_train, y_train)

best_model_index = grid_search_svm.best_index_
best_accuracy = grid_search_svm.cv_results_['mean_test_accuracy'][best_model_index]

# Display results
print('F1-Score Performance:')
print(f'''\n  - Baseline F1 score: {results['Support Vector Machine']['mean_f1']:.4f}''')
print(f'  - Tuned F1 score:    {grid_search_svm.best_score_:.4f}')

print('\nAccuracy Performance:')
print(f'''  - Baseline Accuracy: {results['Support Vector Machine']['mean_accuracy']:.4f}''')
print(f'  - Accuracy of Tuned Model: {best_accuracy:.4f}')

best_svm = grid_search_svm.best_estimator_

Starting Hyperparameter Tuning for Support Vector Machine...
F1-Score Performance:

  - Baseline F1 score: 0.7647
  - Tuned F1 score:    0.7819

Accuracy Performance:
  - Baseline Accuracy: 0.8361
  - Accuracy of Tuned Model: 0.8384


## 4.2. Hyperparameter Tuning Gradient Boosting

In [150]:
gb = GradientBoostingClassifier(random_state=seed)

# Create tuning pipeline
tuning_pipeline = Pipeline(steps=[
    ('preprocessor', grand_pipeline),
    ('classifier', gb)
])

# Define parameter grid
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__max_depth': [3, 4, 5],
    'classifier__subsample': [0.7, 0.9, 1.0]
}

# Create and run GridSearchCV object
grid_search_gb = GridSearchCV(
    tuning_pipeline, 
    param_grid,
    cv=5, 
    scoring={'f1': 'f1', 'accuracy': 'accuracy'},
    refit='f1',
    n_jobs=1
)

# Terminal comments
print('Starting Hyperparameter Tuning for Gradient Boosting...')

# Run tuning
grid_search_gb.fit(X_train, y_train)

best_model_index = grid_search_gb.best_index_
best_accuracy = grid_search_gb.cv_results_['mean_test_accuracy'][best_model_index]

# Display results
print('F1-Score Performance:')
print(f'''- Baseline F1 score: {results['Gradient Boosting']['mean_f1']:.4f}''')
print(f'  - Tuned F1 score:    {grid_search_gb.best_score_:.4f}')

print('\nAccuracy Performance:')
print(f'''  - Baseline Accuracy: {results['Gradient Boosting']['mean_accuracy']:.4f}''')
print(f'  - Accuracy of Tuned Model: {best_accuracy:.4f}')

best_gb = grid_search_gb.best_estimator_

Starting Hyperparameter Tuning for Gradient Boosting...
F1-Score Performance:
- Baseline F1 score: 0.7595
  - Tuned F1 score:    0.7775

Accuracy Performance:
  - Baseline Accuracy: 0.8272
  - Accuracy of Tuned Model: 0.8417


After hyperparameter tuning, both models showed marked improvement. The final cross-validation scores are as follows:

| Model                  | Metric     | Baseline Score | Tuned Score | Change     |
| :--------------------- | :--------- | :------------- | :---------- | :--------- |
| **Support Vector Machine** | **F1-Score** | **0.7647**     | **0.7819**  | **+0.0172**|
|                        | Accuracy   | 0.8361         | 0.8384      | +0.0023    |
| **Gradient Boosting**      | F1-Score   | 0.7595         | 0.7775      | +0.0180    |
|                        | **Accuracy**   | **0.8272**     | **0.8417**  | **+0.0145**|

### Key Findings

- The **Support Vector Machine (SVM)** achieved the highest F1-score, which was our primary metric for optimization.
- **Gradient Boosting** saw the largest overall improvement and now holds the highest accuracy.

While the tuned models granted some better results, I have two possible ideas to further improvement.

1. **XGBoost**: A more efficient and powerful implementation of Gradient Boosting, widely known as a go-to algorithm for machine learning competitions. It offers key advantages over the standard implementation, including advanced regularization to improve accuracy by preventing overfitting, and superior support for parallel processing to significantly reduce training times.

2. **Voting Classifier**: A Voting Classifier is an ensemble method that combines the predictions from multiple models to generate a more robust final prediction.In our case, we can take our two best-tuned models (Support Vector Machine and Gradient Boosting) and average their prediction probabilities. Different models learn in different ways and therefore make different types of errors. By combining them, we can often cancel out these individual mistakes, leading to a more accurate and reliable outcome. The key to a successful ensemble is model diversity, for this reason, we will combine our SVM and Gradient Boosting models, as they have fundamentally different structures. Combining two similar models, like Gradient Boosting and XGBoost, would be less effective as they are likely to make the same kinds of errors.

## 5. Exploring further Improvements with XGBoost

---

In this section we will investigate if XGBoost provides any significant gain in performance compared to our previous two best models (SVM and Gradient Boosting).

### 5.1.


Lets start by making a baseline XGBoost model, before diving into hyperparameter tuning.

In [151]:
from xgboost import XGBClassifier

# Define models
xgb = XGBClassifier(random_state=seed)
model_name = 'XGBoost'

scoring_metrics = ['f1', 'accuracy']

print('Starting baseline-evaluation of XGBoost...\n')

# Train model using the pre-constructed pipeline
full_pipeline = Pipeline(steps=[
    ('preprocessor', grand_pipeline),
    ('classifier', xgb)
])

cv_results = cross_validate(
    full_pipeline, 
    X_train, 
    y_train, 
    cv=5, 
    scoring=scoring_metrics,
    n_jobs=1,  
    error_score='raise'
)

# Store and print the results for both metrics
xgb_mean_f1 = np.mean(cv_results['test_f1'])
xgb_std_f1 = np.std(cv_results['test_f1'])
xgb_mean_accuracy = np.mean(cv_results['test_accuracy'])
xgb_std_accuracy = np.std(cv_results['test_accuracy'])

print('F1-Score Performance:')
print(f'''- Mean F1-Score: {xgb_mean_f1:.4f}''')
print(f'  - Std F1-Score:    {xgb_std_f1:.4f}')

print('\nAccuracy Performance:')
print(f'''  - Mean Accuracy: {xgb_mean_accuracy:.4f}''')
print(f'  - Std Accuracy: {xgb_std_accuracy:.4f}')

Starting baseline-evaluation of XGBoost...

F1-Score Performance:
- Mean F1-Score: 0.7585
  - Std F1-Score:    0.0247

Accuracy Performance:
  - Mean Accuracy: 0.8272
  - Std Accuracy: 0.0081


The baseline evaluation for XGBoost is interesting. Its initial performance is nearly identical to our baseline Gradient Boosting model, with a mean F1-Score of **0.7585** and accuracy of **0.8272**.

So far we do not see any improvement compared to our other models, however XGBoost performance gain often comes from its tuning capabilities and regularization parameters, which we have not utilized yet. The next step is therefore to proceed with hyperparameter tuning to see if we can squeeze some extra performance out.

### 5.2. Hyperparameter Tuning XGBoost

**NOTE:** The cell block below can take very long to run. Especially since I have chose to set n_jobs=1 to avoid warnings for the presentation. To speed up set n_jobs=-1, this allows the model to utilize multiple cores at once, depending on your processor it might speed it up by 10x. 

In [152]:
# Best performing model from base evaluation
xgb = XGBClassifier(random_state=seed, eval_metric='logloss')

# Create tuning pipeline
tuning_pipeline = Pipeline(steps=[
    ('preprocessor', grand_pipeline),
    ('classifier', xgb)
])

# Define parameter grid
param_grid_xgb = {
    'classifier__n_estimators': [100, 200, 400],
    'classifier__max_depth': [3, 4, 5],
    'classifier__learning_rate': [0.01, 0.05, 0.1],  
    'classifier__subsample': [0.7, 0.9, 1.0],
    'classifier__colsample_bytree': [0.7, 0.9, 1.0],
    'classifier__reg_alpha': [0, 0.005, 0.01],   
    'classifier__reg_lambda': [0.1, 1, 10]     
}

# Create and run GridSearchCV object
grid_search_xgb = GridSearchCV(
    tuning_pipeline, 
    param_grid_xgb,
    cv=5, 
    scoring={'f1': 'f1', 'accuracy': 'accuracy'},
    refit='f1',
    n_jobs=1
)

# Terminal comments
print('Starting Hyperparameter Tuning for XGBoost...')

# Run tuning
grid_search_xgb.fit(X_train, y_train)

best_model_index = grid_search_xgb.best_index_
best_accuracy = grid_search_xgb.cv_results_['mean_test_accuracy'][best_model_index]

# Display results
print('F1-Score Performance:')
print(f'''\n  - Baseline F1 score: {xgb_mean_f1:.4f}''')
print(f'  - Tuned F1 score:    {grid_search_xgb.best_score_:.4f}')

print('\nAccuracy Performance:')
print(f'''  - Baseline Accuracy: {xgb_mean_accuracy:.4f}''')
print(f'  - Accuracy of Tuned Model: {best_accuracy:.4f}')

best_xgb = grid_search_xgb.best_estimator_

Starting Hyperparameter Tuning for XGBoost...
F1-Score Performance:

  - Baseline F1 score: 0.7585
  - Tuned F1 score:    0.7858

Accuracy Performance:
  - Baseline Accuracy: 0.8272
  - Accuracy of Tuned Model: 0.8451


After hyperparameter tuning XGBoost is now the best performing model. It outperforms both Gradient Boost in mean accuracy and SVM in mean F1-Score.

## 6. Voting Classifier

---

To potentially gain a final increase in performance we will now make an ensemble model. As mentioned earlier we will use a `Voting Classifier` to combine predictions from our current best performing models, with the intention of minimizing individual errors and producing a more robust end-model. The classifier will be configured to use soft voting, a method where we average the prediction probabilities from the models to make a final prediction. This method is generally superior to hard voting as it leverages the confidence level of each model.

At this point we have three contenders, SVM, XGB, and Gradient Boost. Obviously it is possible to combine all three, however since XGB and Gradient Boost rely on the same underlying algorithm they are likely to make the same mistakes, potentially outvoting SVM. Thus, we will move forward with only SVM and XGB, as XGB was in total the best performing model, and is a better fit then Gradient Boost. 


### 6.1. Ensemble Model

In [153]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_validate

# Retrieve best tuned models
tuned_svm = best_svm.named_steps['classifier']
tuned_xgb = best_xgb.named_steps['classifier']

# Combine models
voting_clf = VotingClassifier(estimators=[
    ('svm', tuned_svm),
    ('xgb', tuned_xgb)
],
voting='soft'
)

# Create final pipeline
ensemble_pipeline = Pipeline(steps=[
    ('preprocessor', grand_pipeline),
    ('ensemble', voting_clf)
])

# Terminal output
print('Evaluating Voting Classifier Ensemble...')

scoring_metrics = ['f1', 'accuracy']

cv_results_ensemble = cross_validate(
    ensemble_pipeline,
    X_train, 
    y_train, 
    cv=5, 
    scoring=scoring_metrics,
    n_jobs=1
)

# Display results
mean_f1_ensemble = np.mean(cv_results_ensemble['test_f1'])
mean_accuracy_ensemble = np.mean(cv_results_ensemble['test_accuracy'])

print(f"  Mean F1 Score: {mean_f1_ensemble:.4f}")
print(f"  Mean Accuracy: {mean_accuracy_ensemble:.4f}")

Evaluating Voting Classifier Ensemble...
  Mean F1 Score: 0.7806
  Mean Accuracy: 0.8440


The results of the ensemble model is worse then XGBoost in both metrics. This indicates that SVM does capture any patterns that XGBoost already does not capture better itself. 

### 6.2. Interpreting the Winning Model: Feature Importance

Throughout this project, a key objective has been to understand our data and explain our results. Now that we have identified XGBoost as the best-performing model, we must also analyze its internal logic to understand how it separates survivors from non-survivors.
XGBoost is a tree-based model. This means it makes predictions by traversing down a series of decision "trees," starting from a root and ending at a leaf. Each step down the tree is based on a "question" about a specific feature (e.g., "Is Sex == 'male'?"). The most important question, the ones that best split the data, are placed at the top of the trees, while smaller details are handled in the lower layers.
A key advantage of this structure is that we can directly see which questions the model deemed most important overall. This is called feature importance, and a primary reason why XGBoost and other decision tree-based models are so widely used when we need not only good accuracy, but also interpretable results.

This interpretability stands in contrast to the Support Vector Machine model we also evaluated. The SVM, particularly with its non-linear kernel, largely functions as a "black box," meaning there is no direct or intuitive way to understand the results of its parameter tuning.
While our tuned SVM was a strong contender for the best model, its lack of clear interpretability would have made it a poor choice for the final analysis, as understanding the model's logic was a key objective of this project.

In [154]:
# get classifier from pipeline
best_xgb_classifier = best_xgb.named_steps['classifier']

# get preprocesser from the pipeline
best_xgb_preprocesser = best_xgb.named_steps['preprocessor']

# get feature names from the ColumnTransformer
feature_names = best_xgb_preprocesser.named_steps['preprocessing'].get_feature_names_out()

# get feature importance
feature_importance = best_xgb_classifier.feature_importances_

# merge feature names with importance
df_feat_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
}).sort_values(by='Importance', ascending=False).T

display(df_feat_importance)

,12,37,9,14,1,33,23,17,27,31,...,13,25,15,35,29,36,3,20,21,22
Feature,cat__Title_feat_Mr.,zone_binned__Zone_binned_Unknown,cat__Pclass_3,cat__Title_feat_Rare,family_survival_rate__FamilySurvivalRate_feat,zone_binned__Zone_binned_Q5,cat__Deck_feat_U,cat__Deck_feat_C,age_binned__Age_binned_Senior,zone_binned__Zone_binned_Q3,...,cat__Title_feat_Mrs.,age_binned__Age_binned_Child,cat__Deck_feat_A,zone_binned__Zone_binned_Q7,zone_binned__Zone_binned_Q1,zone_binned__Zone_binned_Q8,cat__Sex_male,cat__Deck_feat_F,cat__Deck_feat_G,cat__Deck_feat_T
Importance,0.469521,0.082511,0.078042,0.051583,0.032279,0.026897,0.019272,0.01713,0.01689,0.015728,...,0.005199,0.005054,0.004573,0.004465,0.003894,0.001223,0.0,0.0,0.0,0.0


The feature importance analysis reveals a clear and logical hierarchy of predictors that the model used to determine survival.

Unsurprisingly, the title `Mr.` was the single largest factor (**0.47 importance**). This aligns perfectly with our EDA findings, where adult men had the lowest survival rate. The Mr. title effectively captures this high-risk demographic, allowing the model to be highly confident about the outcome for this group from a single feature.

Reinforcing the theme of socio-economic status, the next most important features were `Zone_unknown` and `Pclass_3`. `The Zone_unknown` category, which largely represents passengers in 3rd class without a registered cabin, was a powerful predictor. The model likely learned that this was correlated with both the passengers' poverty and their physical location on the ship, which may have had worse access to lifeboats. The high importance of `Pclass_3` directly confirms this finding.

Crucially, our custom-engineered feature, `FamilySurvivalRate_feat`, ranked as the fifth most predictive feature. This is a significant result, as it validates the work done to create it. It shows that a passengers chance of survival is related to their other family members survival. This could be an indication of that family groups sticked together during crisis, thus surviving together.

**A Note on Features with Zero Importance:** Interestingly, the `Sex` feature received a score of zero on importance by our model, in other words, it had no affect on the model's final prediction. This may seem counterintuitive, as our EDA clearly showed that `Sex` was an extremely strong predictor, with females having a much higher survival rate then males. However, this is not a fault in the model, but a sign that the model has properly identified feature redundancy and multicollinearity. The model understood the titles `Mr` and `Mrs` captures the same relationship then `Sex`, but more detailed and accurately. `Title` also captures social-, marriage-status and age, thus making `Sex` redundant. This also applies to other features (not all),  with an importance of zero.

## 7. Kaggle Customization 

---
This project had a twofold objective: creating a competitive model for the Kaggle leaderboard, while also providing an understandable model that focused not just on high accuracy, but a high F1-score.

Due to the nature of the data, a good model should not only achieve a high overall accuracy but must also be able to distinguish the survivors from the rest. This is why the F1-score was a critical metric throughout our evaluation.

So far, we have chosen the best model based on the F1-score, for the reasons mentioned above. However, the Kaggle leaderboard is judged solely on accuracy.
Therefore, to maximize our competition score, we will perform one final hyperparameter search on our selected model, XGBoost. For this last step, we will optimize for accuracy instead of the F1-score, ensuring our model is aligned with the competition's evaluation metric. This newly tuned model will then be trained on the entire dataset to make our final predictions.

### 7.1. Final Hyperparameter Tuning for Accuracy

In [155]:
# Best performing model from base evaluation
xgb = XGBClassifier(random_state=seed, eval_metric='logloss')

# Create tuning pipeline
tuning_pipeline = Pipeline(steps=[
    ('preprocessor', grand_pipeline),
    ('classifier', xgb)
])

# Define parameter grid
param_grid_xgb = {
    'classifier__n_estimators': [100, 200, 400],
    'classifier__max_depth': [3, 4, 5],
    'classifier__learning_rate': [0.01, 0.05, 0.1],  
    'classifier__subsample': [0.7, 0.9, 1.0],
    'classifier__colsample_bytree': [0.7, 0.9, 1.0],
    'classifier__reg_alpha': [0, 0.005, 0.01],   
    'classifier__reg_lambda': [0.1, 1, 10]     
}

# Create and run GridSearchCV object
grid_search_xgb = GridSearchCV(
    tuning_pipeline, 
    param_grid_xgb,
    cv=5, 
    scoring={'accuracy': 'accuracy'},
    refit='accuracy',
    n_jobs=1
)

# Terminal comments
print('Starting Hyperparameter Tuning for Accuracy XGBoost...')

# Run tuning
grid_search_xgb.fit(X_train, y_train)

best_model_index = grid_search_xgb.best_index_
best_accuracy = grid_search_xgb.cv_results_['mean_test_accuracy'][best_model_index]

# Display results
print('\nAccuracy Performance:')
print(f'''  - Baseline Accuracy: {xgb_mean_accuracy:.4f}''')
print(f'  - Accuracy of Tuned Model: {best_accuracy:.4f}')

best_acc_xgb = grid_search_xgb.best_estimator_

Starting Hyperparameter Tuning for Accuracy XGBoost...

Accuracy Performance:
  - Baseline Accuracy: 0.8272
  - Accuracy of Tuned Model: 0.8485


We were able to achieve a slightly higher mean accuracy compared to previously when we were focused on F1-Score.

The last step and final step is to train the model on the whole train dataset, before we make our final predictions for Kaggle.

### 7.2. Training the Final Model and Generating Predictions

In [156]:
# Our best model
final_kaggle_model = best_acc_xgb

# Training model and the whole training dataset
final_kaggle_model.fit(X_train, y_train)

# Making final predictions
kaggle_predictions = final_kaggle_model.predict(X_test)

# Preparing Submission
submission = pd.DataFrame({
    'PassengerId': X_test.index,
    'Survived': kaggle_predictions
})

# Saving submission as csv
submission.to_csv('../titanic_submission.csv', index=False)

print(submission.shape)
print(submission.head())

(418, 2)
   PassengerId  Survived
0          892         0
1          893         1
2          894         0
3          895         0
4          896         1


### 7.3.

After submitting our final predictions, our model achieved a score of 0.80622, placing it at rank 276 on the leaderboard at the time of submission.

This score is approximately 4 percentage points lower than our final cross-validation accuracy (0.8417). This slight drop is expected and likely reflects a combination of minor overfitting and the natural statistical differences between the training and test sets, rather than a significant flaw in the model.

With further iterations on feature engineering and optimization, it is likely possible to push this score into the 0.82-0.84 range. Nevertheless, a score of 0.80622 is a good result, placing our model firmly within the top 5-10% of all legitimate submissions.